Based on DCGAN from here:

https://github.com/roatienza/Deep-Learning-Experiments/blob/master/Experiments/Tensorflow/GAN/dcgan_mnist.py

Other sources:

https://github.com/jacobgil/keras-dcgan

In [1]:
'''
DCGAN on MNIST using Keras
Author: Rowel Atienza
Project: https://github.com/roatienza/Deep-Learning-Experiments
Dependencies: tensorflow 1.0 and keras 2.0
Usage: python3 dcgan_mnist.py
'''

import numpy as np
import time
from tensorflow.examples.tutorials.mnist import input_data

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Reshape
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers import LeakyReLU, Dropout
from keras.layers import BatchNormalization
from keras.optimizers import Adam, RMSprop

import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
class ElapsedTimer(object):
    def __init__(self):
        self.start_time = time.time()
    def elapsed(self,sec):
        if sec < 60:
            return str(sec) + " sec"
        elif sec < (60 * 60):
            return str(sec / 60) + " min"
        else:
            return str(sec / (60 * 60)) + " hr"
    def elapsed_time(self):
        print("Elapsed: %s " % self.elapsed(time.time() - self.start_time) )

In [3]:
class DCGAN(object):
    def __init__(self, img_rows=28, img_cols=28, channel=1):

        self.img_rows = img_rows
        self.img_cols = img_cols
        self.channel = channel
        self.D = None   # discriminator
        self.G = None   # generator
        self.AM = None  # adversarial model
        self.DM = None  # discriminator model

    # (W−F+2P)/S+1
    def discriminator(self):
        if self.D:
            return self.D
        self.D = Sequential()
        depth = 64
        dropout = 0.4
        # In: 28 x 28 x 1, depth = 1
        # Out: 14 x 14 x 1, depth=64
        input_shape = (self.img_rows, self.img_cols, self.channel)
        self.D.add(Conv2D(depth*1, 5, strides=2, input_shape=input_shape,\
            padding='same'))
        self.D.add(LeakyReLU(alpha=0.2))
        self.D.add(Dropout(dropout))

        self.D.add(Conv2D(depth*2, 5, strides=2, padding='same'))
        self.D.add(LeakyReLU(alpha=0.2))
        self.D.add(Dropout(dropout))

        self.D.add(Conv2D(depth*4, 5, strides=2, padding='same'))
        self.D.add(LeakyReLU(alpha=0.2))
        self.D.add(Dropout(dropout))

        self.D.add(Conv2D(depth*8, 5, strides=1, padding='same'))
        self.D.add(LeakyReLU(alpha=0.2))
        self.D.add(Dropout(dropout))

        # Out: 1-dim probability
        self.D.add(Flatten())
        self.D.add(Dense(1))
        self.D.add(Activation('sigmoid'))
        self.D.summary()
        return self.D

    def generator(self):
        if self.G:
            return self.G
        self.G = Sequential()
        dropout = 0.4
        depth = 64+64+64+64
        dim = 7
        # In: 100
        # Out: dim x dim x depth
        self.G.add(Dense(dim*dim*depth, input_dim=100))
        self.G.add(BatchNormalization(momentum=0.9))
        self.G.add(Activation('relu'))
        self.G.add(Reshape((dim, dim, depth)))
        self.G.add(Dropout(dropout))

        # In: dim x dim x depth
        # Out: 2*dim x 2*dim x depth/2
        self.G.add(UpSampling2D())
        self.G.add(Conv2DTranspose(int(depth/2), 5, padding='same'))
        self.G.add(BatchNormalization(momentum=0.9))
        self.G.add(Activation('relu'))

        self.G.add(UpSampling2D())
        self.G.add(Conv2DTranspose(int(depth/4), 5, padding='same'))
        self.G.add(BatchNormalization(momentum=0.9))
        self.G.add(Activation('relu'))

        self.G.add(Conv2DTranspose(int(depth/8), 5, padding='same'))
        self.G.add(BatchNormalization(momentum=0.9))
        self.G.add(Activation('relu'))

        # Out: 28 x 28 x 1 grayscale image [0.0,1.0] per pix
        self.G.add(Conv2DTranspose(1, 5, padding='same'))
        self.G.add(Activation('sigmoid'))
        self.G.summary()
        return self.G

    def discriminator_model(self):
        if self.DM:
            return self.DM
        optimizer = RMSprop(lr=0.0002, decay=6e-8)
        self.DM = Sequential()
        self.DM.add(self.discriminator())
        self.DM.compile(loss='binary_crossentropy', optimizer=optimizer,\
            metrics=['accuracy'])
        return self.DM

    def adversarial_model(self):
        if self.AM:
            return self.AM
        optimizer = RMSprop(lr=0.0001, decay=3e-8)
        self.AM = Sequential()
        self.AM.add(self.generator())
        self.AM.add(self.discriminator())
        self.AM.compile(loss='binary_crossentropy', optimizer=optimizer,\
            metrics=['accuracy'])
        return self.AM

In [4]:
class MNIST_DCGAN(object):
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channel = 1

        self.x_train = input_data.read_data_sets("mnist",\
        	one_hot=True).train.images
        self.x_train = self.x_train.reshape(-1, self.img_rows,\
        	self.img_cols, 1).astype(np.float32)

        self.DCGAN = DCGAN()
        self.discriminator =  self.DCGAN.discriminator_model()
        self.adversarial = self.DCGAN.adversarial_model()
        self.generator = self.DCGAN.generator()

    def train(self, train_steps=2000, batch_size=256, save_interval=0):
        noise_input = None
        if save_interval>0:
            noise_input = np.random.uniform(-1.0, 1.0, size=[16, 100])
        for i in range(train_steps):
            images_train = self.x_train[np.random.randint(0,
                self.x_train.shape[0], size=batch_size), :, :, :]
            noise = np.random.uniform(-1.0, 1.0, size=[batch_size, 100])
            images_fake = self.generator.predict(noise)
            x = np.concatenate((images_train, images_fake))
            y = np.ones([2*batch_size, 1])
            y[batch_size:, :] = 0
            d_loss = self.discriminator.train_on_batch(x, y)

            y = np.ones([batch_size, 1])
            noise = np.random.uniform(-1.0, 1.0, size=[batch_size, 100])
            a_loss = self.adversarial.train_on_batch(noise, y)
            log_mesg = "%d: [D loss: %f, acc: %f]" % (i, d_loss[0], d_loss[1])
            log_mesg = "%s  [A loss: %f, acc: %f]" % (log_mesg, a_loss[0], a_loss[1])
            print(log_mesg)
            if save_interval>0:
                if (i+1)%save_interval==0:
                    self.plot_images(save2file=True, samples=noise_input.shape[0],\
                        noise=noise_input, step=(i+1))

    def plot_images(self, save2file=False, fake=True, samples=16, noise=None, step=0):
        filename = 'mnist.png'
        if fake:
            if noise is None:
                noise = np.random.uniform(-1.0, 1.0, size=[samples, 100])
            else:
                filename = "mnist_%d.png" % step
            images = self.generator.predict(noise)
        else:
            i = np.random.randint(0, self.x_train.shape[0], samples)
            images = self.x_train[i, :, :, :]

        plt.figure(figsize=(10,10))
        for i in range(images.shape[0]):
            plt.subplot(4, 4, i+1)
            image = images[i, :, :, :]
            image = np.reshape(image, [self.img_rows, self.img_cols])
            plt.imshow(image, cmap='gray')
            plt.axis('off')
        plt.tight_layout()
        if save2file:
            plt.savefig(filename)
            plt.close('all')
        else:
            plt.show()

In [5]:
if __name__ == '__main__':
    mnist_dcgan = MNIST_DCGAN()
    timer = ElapsedTimer()
    mnist_dcgan.train(train_steps=10000, batch_size=256, save_interval=500)
    timer.elapsed_time()
    mnist_dcgan.plot_images(fake=True)
    mnist_dcgan.plot_images(fake=False, save2file=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting mnist/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting mnist/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting mnist/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting mnist/t10k-labels-idx1-ubyte.gz
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 14, 14, 64)        1664      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7,

38: [D loss: 0.024725, acc: 0.998047]  [A loss: 0.001441, acc: 1.000000]
39: [D loss: 0.032824, acc: 0.998047]  [A loss: 0.003067, acc: 1.000000]
40: [D loss: 0.025561, acc: 1.000000]  [A loss: 0.003965, acc: 1.000000]
41: [D loss: 0.033712, acc: 0.998047]  [A loss: 0.004541, acc: 1.000000]
42: [D loss: 0.043093, acc: 0.998047]  [A loss: 0.040548, acc: 0.988281]
43: [D loss: 0.695135, acc: 0.658203]  [A loss: 6.502824, acc: 0.000000]
44: [D loss: 2.143780, acc: 0.505859]  [A loss: 8.191023, acc: 0.000000]
45: [D loss: 0.300820, acc: 0.923828]  [A loss: 2.088740, acc: 0.023438]
46: [D loss: 0.855108, acc: 0.566406]  [A loss: 6.136967, acc: 0.000000]
47: [D loss: 0.318919, acc: 0.931641]  [A loss: 3.441426, acc: 0.000000]
48: [D loss: 0.690661, acc: 0.583984]  [A loss: 7.943220, acc: 0.000000]
49: [D loss: 0.373503, acc: 0.826172]  [A loss: 1.176194, acc: 0.230469]
50: [D loss: 1.061980, acc: 0.500000]  [A loss: 7.462814, acc: 0.000000]
51: [D loss: 0.477730, acc: 0.705078]  [A loss: 1.9

150: [D loss: 0.561347, acc: 0.605469]  [A loss: 2.292832, acc: 0.000000]
151: [D loss: 0.467412, acc: 0.867188]  [A loss: 0.672100, acc: 0.550781]
152: [D loss: 0.750948, acc: 0.501953]  [A loss: 2.085491, acc: 0.000000]
153: [D loss: 0.476391, acc: 0.859375]  [A loss: 0.775192, acc: 0.390625]
154: [D loss: 0.599649, acc: 0.517578]  [A loss: 1.528679, acc: 0.000000]
155: [D loss: 0.427156, acc: 0.912109]  [A loss: 1.104988, acc: 0.050781]
156: [D loss: 0.498557, acc: 0.630859]  [A loss: 1.937962, acc: 0.000000]
157: [D loss: 0.399610, acc: 0.908203]  [A loss: 1.156844, acc: 0.062500]
158: [D loss: 0.539561, acc: 0.609375]  [A loss: 2.358469, acc: 0.000000]
159: [D loss: 0.457586, acc: 0.851562]  [A loss: 0.685625, acc: 0.539062]
160: [D loss: 0.757207, acc: 0.501953]  [A loss: 2.148816, acc: 0.000000]
161: [D loss: 0.488889, acc: 0.837891]  [A loss: 0.714659, acc: 0.476562]
162: [D loss: 0.650433, acc: 0.523438]  [A loss: 1.581339, acc: 0.000000]
163: [D loss: 0.467863, acc: 0.845703]

261: [D loss: 0.586083, acc: 0.767578]  [A loss: 0.646375, acc: 0.625000]
262: [D loss: 0.750961, acc: 0.500000]  [A loss: 1.854499, acc: 0.000000]
263: [D loss: 0.665489, acc: 0.558594]  [A loss: 0.616258, acc: 0.699219]
264: [D loss: 0.819947, acc: 0.492188]  [A loss: 1.209861, acc: 0.027344]
265: [D loss: 0.594486, acc: 0.681641]  [A loss: 1.058092, acc: 0.042969]
266: [D loss: 0.627030, acc: 0.648438]  [A loss: 1.233855, acc: 0.007812]
267: [D loss: 0.609148, acc: 0.691406]  [A loss: 0.955643, acc: 0.093750]
268: [D loss: 0.628467, acc: 0.583984]  [A loss: 1.293141, acc: 0.000000]
269: [D loss: 0.609744, acc: 0.675781]  [A loss: 1.012945, acc: 0.066406]
270: [D loss: 0.645847, acc: 0.583984]  [A loss: 1.458077, acc: 0.000000]
271: [D loss: 0.621887, acc: 0.695312]  [A loss: 0.710482, acc: 0.500000]
272: [D loss: 0.721667, acc: 0.513672]  [A loss: 1.769747, acc: 0.000000]
273: [D loss: 0.642980, acc: 0.574219]  [A loss: 0.596137, acc: 0.746094]
274: [D loss: 0.736645, acc: 0.505859]

372: [D loss: 0.645465, acc: 0.646484]  [A loss: 0.870910, acc: 0.121094]
373: [D loss: 0.670071, acc: 0.562500]  [A loss: 0.981300, acc: 0.050781]
374: [D loss: 0.664055, acc: 0.619141]  [A loss: 0.914420, acc: 0.074219]
375: [D loss: 0.661405, acc: 0.578125]  [A loss: 0.936241, acc: 0.058594]
376: [D loss: 0.650994, acc: 0.599609]  [A loss: 0.993941, acc: 0.042969]
377: [D loss: 0.663548, acc: 0.611328]  [A loss: 0.982531, acc: 0.054688]
378: [D loss: 0.671103, acc: 0.611328]  [A loss: 0.968796, acc: 0.074219]
379: [D loss: 0.657306, acc: 0.599609]  [A loss: 1.130643, acc: 0.000000]
380: [D loss: 0.651839, acc: 0.628906]  [A loss: 0.745779, acc: 0.394531]
381: [D loss: 0.699119, acc: 0.509766]  [A loss: 1.320012, acc: 0.000000]
382: [D loss: 0.664011, acc: 0.599609]  [A loss: 0.585926, acc: 0.828125]
383: [D loss: 0.756465, acc: 0.500000]  [A loss: 1.222411, acc: 0.000000]
384: [D loss: 0.690725, acc: 0.527344]  [A loss: 0.742004, acc: 0.367188]
385: [D loss: 0.687756, acc: 0.527344]

...

4778: [D loss: 0.690451, acc: 0.560547]  [A loss: 0.722611, acc: 0.500000]
4779: [D loss: 0.732265, acc: 0.500000]  [A loss: 0.995128, acc: 0.097656]
4780: [D loss: 0.704379, acc: 0.507812]  [A loss: 0.706288, acc: 0.511719]
4781: [D loss: 0.733225, acc: 0.486328]  [A loss: 0.947319, acc: 0.078125]
4782: [D loss: 0.705245, acc: 0.505859]  [A loss: 0.727916, acc: 0.421875]
4783: [D loss: 0.721112, acc: 0.505859]  [A loss: 0.935664, acc: 0.125000]
4784: [D loss: 0.689551, acc: 0.511719]  [A loss: 0.700995, acc: 0.496094]
4785: [D loss: 0.712578, acc: 0.513672]  [A loss: 0.899392, acc: 0.152344]
4786: [D loss: 0.699620, acc: 0.515625]  [A loss: 0.747190, acc: 0.414062]
4787: [D loss: 0.723363, acc: 0.533203]  [A loss: 0.952809, acc: 0.101562]
4788: [D loss: 0.699149, acc: 0.513672]  [A loss: 0.750278, acc: 0.390625]
4789: [D loss: 0.728418, acc: 0.509766]  [A loss: 0.896702, acc: 0.164062]
4790: [D loss: 0.716908, acc: 0.466797]  [A loss: 0.761898, acc: 0.371094]
4791: [D loss: 0.714801, 

4888: [D loss: 0.706524, acc: 0.509766]  [A loss: 0.732744, acc: 0.441406]
4889: [D loss: 0.719153, acc: 0.496094]  [A loss: 0.927120, acc: 0.132812]
4890: [D loss: 0.707329, acc: 0.509766]  [A loss: 0.775643, acc: 0.347656]
4891: [D loss: 0.701695, acc: 0.515625]  [A loss: 0.847207, acc: 0.250000]
4892: [D loss: 0.706587, acc: 0.503906]  [A loss: 0.806316, acc: 0.300781]
4893: [D loss: 0.694721, acc: 0.562500]  [A loss: 0.838320, acc: 0.250000]
4894: [D loss: 0.701903, acc: 0.544922]  [A loss: 0.833021, acc: 0.269531]
4895: [D loss: 0.705863, acc: 0.511719]  [A loss: 0.785439, acc: 0.343750]
4896: [D loss: 0.693882, acc: 0.539062]  [A loss: 0.842985, acc: 0.246094]
4897: [D loss: 0.708280, acc: 0.501953]  [A loss: 0.799394, acc: 0.332031]
4898: [D loss: 0.711041, acc: 0.505859]  [A loss: 0.839324, acc: 0.230469]
4899: [D loss: 0.703068, acc: 0.548828]  [A loss: 0.794135, acc: 0.359375]
4900: [D loss: 0.711162, acc: 0.523438]  [A loss: 0.881914, acc: 0.203125]
4901: [D loss: 0.697399, 

4998: [D loss: 0.730165, acc: 0.494141]  [A loss: 0.948670, acc: 0.140625]
4999: [D loss: 0.705492, acc: 0.525391]  [A loss: 0.728893, acc: 0.445312]
5000: [D loss: 0.724210, acc: 0.523438]  [A loss: 0.906220, acc: 0.160156]
5001: [D loss: 0.701326, acc: 0.498047]  [A loss: 0.737491, acc: 0.402344]
5002: [D loss: 0.736570, acc: 0.498047]  [A loss: 1.053423, acc: 0.074219]
5003: [D loss: 0.708079, acc: 0.519531]  [A loss: 0.730882, acc: 0.410156]
5004: [D loss: 0.724187, acc: 0.472656]  [A loss: 0.804143, acc: 0.304688]
5005: [D loss: 0.714247, acc: 0.494141]  [A loss: 0.772789, acc: 0.371094]
5006: [D loss: 0.713340, acc: 0.521484]  [A loss: 0.885586, acc: 0.167969]
5007: [D loss: 0.716703, acc: 0.500000]  [A loss: 0.759693, acc: 0.394531]
5008: [D loss: 0.710930, acc: 0.525391]  [A loss: 0.865090, acc: 0.210938]
5009: [D loss: 0.699928, acc: 0.541016]  [A loss: 0.773738, acc: 0.398438]
5010: [D loss: 0.711650, acc: 0.531250]  [A loss: 0.983227, acc: 0.089844]
5011: [D loss: 0.719101, 

5108: [D loss: 0.697078, acc: 0.541016]  [A loss: 0.773083, acc: 0.382812]
5109: [D loss: 0.715489, acc: 0.511719]  [A loss: 0.861382, acc: 0.222656]
5110: [D loss: 0.711260, acc: 0.521484]  [A loss: 0.741900, acc: 0.375000]
5111: [D loss: 0.712996, acc: 0.515625]  [A loss: 0.974405, acc: 0.085938]
5112: [D loss: 0.680639, acc: 0.589844]  [A loss: 0.716265, acc: 0.492188]
5113: [D loss: 0.722368, acc: 0.519531]  [A loss: 0.904281, acc: 0.156250]
5114: [D loss: 0.691239, acc: 0.546875]  [A loss: 0.735065, acc: 0.421875]
5115: [D loss: 0.721464, acc: 0.507812]  [A loss: 0.831378, acc: 0.238281]
5116: [D loss: 0.691711, acc: 0.513672]  [A loss: 0.776260, acc: 0.359375]
5117: [D loss: 0.716093, acc: 0.521484]  [A loss: 0.860890, acc: 0.203125]
5118: [D loss: 0.694990, acc: 0.529297]  [A loss: 0.791810, acc: 0.308594]
5119: [D loss: 0.707287, acc: 0.519531]  [A loss: 0.821022, acc: 0.277344]
5120: [D loss: 0.707828, acc: 0.501953]  [A loss: 0.812406, acc: 0.265625]
5121: [D loss: 0.696471, 

KeyboardInterrupt: 